In [1]:
# --- Cell 1: Imports and Configuration ---

import requests
import pandas as pd
import logging
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
from tqdm.notebook import tqdm
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# --- Load environment variables from .env file ---
# Ensure you have a .env file in the same directory with FEEGOW_TOKEN="your_token_here"
load_dotenv()

# --- API and Header Configuration ---
FEEGOW_URL = "https://api.feegow.com/v1/api"
FEEGOW_TOKEN = os.getenv("FEEGOW_TOKEN")

HEADERS = {
    "x-access-token": FEEGOW_TOKEN,
    "Content-Type": "application/json"
}

# --- Data Cleaning Configuration ---
# List of columns to remove from the final DataFrame. Edit this list as needed.
COLUMNS_TO_DROP = [
    'documento', 'endereco', 'numero', 'complemento', 'foto',
    'matricula', 'telefones', 'celulares', 'documentos', 'email'
]

# --- Setup structured logging for Notebook ---
# This check prevents duplicate handlers if you re-run the cell.
logger = logging.getLogger()
if not logger.handlers:
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )

In [3]:
# --- Cell 2: API Functions and Logic ---

def feegow_api_get(
    session: requests.Session, 
    endpoint: str, 
    params: Optional[Dict[str, Any]] = None
) -> Optional[Dict[str, Any]]:
    """Makes a GET request to a Feegow API endpoint using a session object."""
    try:
        response = session.get(f"{FEEGOW_URL}{endpoint}", params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        logging.error(f"HTTP error for endpoint '{endpoint}': {http_err} - Response: {http_err.response.text}")
    except requests.exceptions.RequestException as req_err:
        logging.error(f"Request error for endpoint '{endpoint}': {req_err}")
    except ValueError: # Catches JSON decoding errors
        logging.error(f"JSON decode error for endpoint '{endpoint}'. Response content might not be valid JSON.")
    return None

def get_patient_list(session: requests.Session, limit_per_page: int) -> List[Dict[str, Any]]:
    """Fetches a list of all patients (summary data) from the /patient/list endpoint."""
    all_patients_summary: List[Dict[str, Any]] = []
    current_offset = 0
    logging.info("Starting to fetch patient list summaries...")
    
    while True:
        logging.info(f"Fetching patient list: offset {current_offset}, limit {limit_per_page}...")
        params = {"limit": limit_per_page, "offset": current_offset}
        data = feegow_api_get(session, "/patient/list", params=params)
        
        if data and data.get("success") and "content" in data:
            patients_batch = data["content"]
            if not patients_batch:
                logging.info("No more patient summaries found. Reached the end.")
                break
            all_patients_summary.extend(patients_batch)
            if len(patients_batch) < limit_per_page:
                logging.info("Last page of patient summaries reached.")
                break
            current_offset += limit_per_page
        else:
            logging.warning(f"Failed to fetch patient list or no content at offset {current_offset}. API Response: {data}")
            break
            
    logging.info(f"Fetched a total of {len(all_patients_summary)} patient summaries.")
    return all_patients_summary

def fetch_single_patient_detail(session: requests.Session, patient_summary: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    """Fetches details for a single patient and merges specified fields from the summary."""
    patient_id = patient_summary.get("patient_id")
    if not patient_id:
        logging.warning(f"Patient summary is missing 'patient_id'. Summary: {patient_summary}")
        return None

    params = {"paciente_id": patient_id, "photo": "false"}
    data = feegow_api_get(session, "/patient/search", params=params)
    
    if data and data.get("success") and "content" in data:
        detailed_data = data["content"]
        detailed_data['criado_em'] = patient_summary.get('criado_em')
        detailed_data['alterado_em'] = patient_summary.get('alterado_em')
        
        if 'id' in detailed_data and 'patient_id' not in detailed_data:
             detailed_data['patient_id'] = detailed_data['id']
        elif 'patient_id' not in detailed_data:
             detailed_data['patient_id'] = patient_id # Fallback
        return detailed_data
    else:
        logging.error(f"Failed to fetch details for patient_id {patient_id}. API Response: {data}")
        return None
 
def get_all_patients_with_full_details(
    max_workers: int = 10,
    limit_per_page: int = 3500,
    columns_to_drop: Optional[List[str]] = None,
    output_csv_basename: str = "feegow_all_patients_detailed"
) -> pd.DataFrame:
    """
    Fetches all patients and their detailed data concurrently, saves to a timestamped CSV,
    and returns a Pandas DataFrame.
    """
    if not FEEGOW_TOKEN or FEEGOW_TOKEN == "your_token_here":
        logging.critical("Critical Error: FEEGOW_TOKEN is not configured. Please set it in your .env file.")
        return pd.DataFrame()

    all_detailed_data: List[Dict[str, Any]] = []
    
    with requests.Session() as session:
        session.headers.update(HEADERS)
        
        patient_summaries = get_patient_list(session, limit_per_page)
        if not patient_summaries:
            logging.warning("No patient summaries were fetched. Exiting.")
            return pd.DataFrame()
            
        total_patients = len(patient_summaries)
        logging.info(f"Fetching full details for {total_patients} patients using up to {max_workers} parallel workers...")

        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_summary = {executor.submit(fetch_single_patient_detail, session, summary): summary for summary in patient_summaries}
            
            for future in tqdm(as_completed(future_to_summary), total=total_patients, desc="Fetching Patient Details"):
                result = future.result()
                if result:
                    all_detailed_data.append(result)

    if not all_detailed_data:
        logging.warning("\nNo detailed patient data could be fetched successfully.")
        return pd.DataFrame()

    logging.info(f"\nSuccessfully fetched full details for {len(all_detailed_data)} out of {total_patients} patients.")
    
    df = pd.DataFrame(all_detailed_data)
    logging.info("Consolidated patient data into a DataFrame.")
    
    # Clean the DataFrame by dropping specified columns
    if columns_to_drop is None:
        columns_to_drop = []
    # Ensure we only drop columns that actually exist in the DataFrame
    existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]
    df_cleaned = df.drop(columns=existing_columns_to_drop)
    logging.info(f"Dropped the following columns: {existing_columns_to_drop}")
    
    # Generate timestamped filename and save to CSV
    timestamp = datetime.now().strftime("%Y-%m-%d")
    output_csv_filename = f"{output_csv_basename}_{timestamp}.csv"
    
    try:
        df_cleaned.to_csv(output_csv_filename, index=False, encoding='utf-8-sig')
        logging.info(f"DataFrame successfully saved to '{output_csv_filename}'")
    except Exception as e:
        logging.error(f"Error saving DataFrame to CSV '{output_csv_filename}': {e}")
        
    return df

In [4]:
# --- Cell 3: Execution ---

# You can adjust the number of parallel workers and page limit depending on your network
# and the API's rate-limiting policies. 10-20 workers is often a good starting point.
# The columns to drop are now configured in Cell 1.
detailed_patient_df = get_all_patients_with_full_details(
    max_workers=10,
    limit_per_page=4000,
    columns_to_drop=COLUMNS_TO_DROP
)

# Display the results directly in the notebook
if not detailed_patient_df.empty:
    print("\n--- Sample of Original Fetched Data (Before Column Drop) ---")
    display(detailed_patient_df.head())
    print("\n--- DataFrame Info ---")
    detailed_patient_df.info()
else:
    print("\nProcess finished, but the resulting DataFrame is empty.")

2025-06-27 18:58:08 - INFO - Starting to fetch patient list summaries...
2025-06-27 18:58:08 - INFO - Fetching patient list: offset 0, limit 4000...
2025-06-27 18:58:12 - INFO - Fetching patient list: offset 4000, limit 4000...
2025-06-27 18:58:15 - INFO - Fetching patient list: offset 8000, limit 4000...
2025-06-27 18:58:17 - INFO - Fetching patient list: offset 12000, limit 4000...
2025-06-27 18:58:20 - INFO - Fetching patient list: offset 16000, limit 4000...
2025-06-27 18:58:23 - INFO - Fetching patient list: offset 20000, limit 4000...
2025-06-27 18:58:26 - INFO - Fetching patient list: offset 24000, limit 4000...
2025-06-27 18:58:29 - INFO - Fetching patient list: offset 28000, limit 4000...
2025-06-27 18:58:31 - INFO - Fetching patient list: offset 32000, limit 4000...
2025-06-27 18:58:35 - INFO - Fetching patient list: offset 36000, limit 4000...
2025-06-27 18:58:35 - INFO - Last page of patient summaries reached.
2025-06-27 18:58:35 - INFO - Fetched a total of 36702 patient su

Fetching Patient Details:   0%|          | 0/36702 [00:00<?, ?it/s]

2025-06-27 19:06:06 - INFO - 
Successfully fetched full details for 36702 out of 36702 patients.
2025-06-27 19:06:07 - INFO - Consolidated patient data into a DataFrame.
2025-06-27 19:06:07 - INFO - Dropped the following columns: ['documento', 'endereco', 'numero', 'complemento', 'foto', 'matricula', 'telefones', 'celulares', 'documentos', 'email']
2025-06-27 19:06:08 - INFO - DataFrame successfully saved to 'feegow_all_patients_detailed_2025-06-27.csv'



--- Sample of Original Fetched Data (Before Column Drop) ---


,id,nome,nome_social,nascimento,sexo,origem_id,tabela_id,documento,endereco,numero,...,foto,telefones,celulares,documentos,email,convenios,programa_de_saude,criado_em,alterado_em,patient_id
0,29675,João Gabriel Coelho de Oliveira,None,11-11-2000,Masculino,None,NaN,273782649,Rua Navarro,325,...,,"[None, None]","[(21) 99162-7878, None]","{'rg': '273782649', 'cpf': '18926599728'}","[gabrielolveira201258@gmail.com, None]",[],[],2021-11-24 20:56:00,2021-11-24 20:56:00,29675
1,37718,Maria Elena Santana Ojeda,None,12-05-1969,Feminino,None,NaN,None,Avenida Paulo de Fronti,730,...,,"[None, None]","[(21) 96943-2227, None]","{'rg': None, 'cpf': '6387876702'}","[melenasantana@hotmaol.com, None]",[],[],2021-11-24 20:56:00,2021-11-24 20:56:00,37718
2,48267,Maria de Fátima Arruda,None,28-01-1958,Feminino,None,NaN,43459544,Rua Soares Cabral,21,...,,"[None, None]","[21966899835, None]","{'rg': '43459544', 'cpf': '2930294744'}","[sonnydani@gmail.com, None]",[],[],2021-11-24 20:56:09,2021-11-29 17:27:08,48267
3,17219,Irênio Alves Pinto,None,13-08-1946,Masculino,None,NaN,None,Rua Tavares Bastos,314,...,,"[None, None]","[(21) 98745-5766, None]","{'rg': None, 'cpf': '33564418768'}","[irenio46@hotmail.com, None]",[],[],2021-11-24 20:56:00,2021-11-24 20:56:00,17219
4,15715,Ana Lucia Alvarez Rodrigues,None,03-11-1967,Feminino,None,NaN,517692,Rua José Bonifácio,140,...,,"[None, None]","[(21) 99945-8692, None]","{'rg': '517692', 'cpf': '96243970744'}","[analucia_arodrigues@hotmail.com, None]",[],[],2021-11-24 21:46:08,2021-11-24 21:46:08,15715



--- DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36702 entries, 0 to 36701
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 36702 non-null  int64  
 1   nome               36702 non-null  object 
 2   nome_social        20823 non-null  object 
 3   nascimento         28529 non-null  object 
 4   sexo               26035 non-null  object 
 5   origem_id          21260 non-null  object 
 6   tabela_id          23175 non-null  float64
 7   documento          22536 non-null  object 
 8   endereco           25233 non-null  object 
 9   numero             23309 non-null  object 
 10  complemento        22590 non-null  object 
 11  bairro             25809 non-null  object 
 12  cidade             23233 non-null  object 
 13  estado             23186 non-null  object 
 14  pais               20410 non-null  float64
 15  observacao         24158 non-null  object 
 16